In [1]:
import pandas as pd
import re
from datetime import datetime
from fetch_process_function import calculate_pnl
'''
不用看，测试文件
data_process.ipynb和data_fetch.ipynb是获取数据以及处理的一个流程，和cal_pnL.ipynb，fetch_process_function.py是同样的作用，后者是将前者的代码整理成一个py文件，方便调用，更加简洁
'''
OUTPUT_FILE = 'C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result'
api_key = "VQAIR728IM4Z8RZKPYBR4ESM5I3WBZK2C1" # my free API key, you can get one at https://etherscan.io/myapikey
base_url = "https://api.etherscan.io/v2/api" # We're using the v2 API 2024/12/12
ADDRESS = "0x2c89a2ee92b9870f55989b4132a58c0e85222d86" # Address of the contract we want to get the source code of
INVALID_ADDRESS = "0x0000000000000000000000000000000000000000"

In [2]:
def combine_group(group):
    # 1.检查 timestamp 是否唯一
    if group['timeStamp'].nunique() != 1:
        print("❌ The timestamps are not consistent.")
        return None

    # 2.检查 token 是否只有两种
    if group['tokenSymbol'].nunique() != 2:
        print("❌ The tokens are not exactly two.")
        return None

    # 3.将涉及 ADDRESS 的交易合并
    merged_data = []
    # token一定只有两种
    for token in group['tokenSymbol'].unique():
        # 筛选当前 token 的交易
        token_group = group[group['tokenSymbol'] == token]

        # 筛选涉及 ADDRESS 的交易:因为本人地址总会出现在from或to中，而且每行都会出现，其余地址即是我们要统计的
        address_tx = token_group[(token_group['from'] == ADDRESS) | (token_group['to'] == ADDRESS)]

        if not address_tx.empty:
            # 累加 actualValue
            total_value = address_tx['ActualValue'].sum()

            # 获取其他地址
            other_addresses = address_tx[address_tx['from'] != ADDRESS]['from'].tolist() + \
                                address_tx[address_tx['to'] != ADDRESS]['to'].tolist()
            other_addresses = '.'.join(sorted(set(other_addresses)))  # 去重并排序

            # 确定 from 和 to
            if address_tx['from'].iloc[0] == ADDRESS:
                from_address = ADDRESS
                to_address = other_addresses
            else:
                from_address = other_addresses
                to_address = ADDRESS

            merged_tx = {
                'dateTime': address_tx['dateTime'].iloc[0],
                'blockNumber': address_tx['blockNumber'].iloc[0],
                'timeStamp': address_tx['timeStamp'].iloc[0],
                'hash': address_tx['hash'].iloc[0],
                'from': from_address,
                'to': to_address,
                'ActualValue': total_value,
                'tokenName': address_tx['tokenName'].iloc[0],
                'tokenSymbol': token
            }
            merged_data.append(merged_tx)

    combined_df = pd.DataFrame(merged_data)
    return combined_df


def highlight_three_records(grouped_df):
    print(f"...Merging {len(grouped_df)} groups of transactions...")
    matched_hashes = []
    combined_hashes_list = []
    for tx_hash, group in grouped_df:
        if len(group) >= 3:
            print(f"Transaction Hash {tx_hash} contains {len(group)} records:")
            matched_hashes.append(tx_hash)
            
            # 1.先检查是不是偶数个 奇数个显然不合理
            if len(group) % 2 != 0:
                print("❌ The number of records is not even.")
            else:
                # 调用 combine_group 检查并合并
                combined_hash = combine_group(group)
                if combined_hash is not None:
                    combined_hashes_list.append(combined_hash)
                    
    if combined_hashes_list:
        combined_hashes_df = pd.concat(combined_hashes_list, ignore_index=True)
    else:
        combined_hashes_df = pd.DataFrame()
    return matched_hashes, combined_hashes_df


In [3]:
global BASE_TOKENS
BASE_TOKENS = {"USDT", "USDC", "USDE", "DAI"}
base_pattern = re.compile('|'.join(BASE_TOKENS))
ADDRESS = ADDRESS.lower()

csv_file = 'C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result/erc20_transfers.csv'
transaction_data = pd.read_csv(csv_file)
transaction_data['dateTime'] = pd.to_datetime(transaction_data['dateTime'])
transaction_data.loc[:, 'ActualValue'] = pd.to_numeric(transaction_data['ActualValue'], errors='coerce')
# 如果不转换为数据的话计算会出错
transaction_data = transaction_data.sort_values(by=['dateTime', 'hash']).reset_index(drop=True)
# make sure the date data is correct

columns_to_keep = [
    'dateTime', 'blockNumber', 'timeStamp', 'hash', 'from', 'to',
    'ActualValue', 'tokenName', 'tokenSymbol'
    ]
filtered_transaction_data = transaction_data[columns_to_keep]
filtered_transaction_data.to_csv('filtered_transaction_data.csv', index=False)
# 清洗数据
filtered_transaction_data = filtered_transaction_data[
    (transaction_data['from'] != INVALID_ADDRESS) &
    (transaction_data['to'] != INVALID_ADDRESS)
]
output_file = 'cleaned_transaction_data.csv'
filtered_transaction_data[columns_to_keep].to_csv(output_file, index=False)

In [5]:
duplicate_hashes = filtered_transaction_data[filtered_transaction_data.duplicated(subset=['hash'], keep=False)]

# check if there are abnoraml data, print it and delete from original data
three_record_hashes, combined_df = highlight_three_records(duplicate_hashes.groupby('hash'))

# delete the dulicate data that are more than 2, so that we can cntinue the calculation
duplicate_hashes = duplicate_hashes[~duplicate_hashes['hash'].isin(three_record_hashes)]

merged_df = pd.concat([duplicate_hashes, combined_df], ignore_index=True)
merged_df = merged_df.sort_values(by='timeStamp').reset_index(drop=True)
merged_df.to_csv('merged_transaction_data.csv', index=False)

...Merging 456 groups of transactions...
Transaction Hash 0x0fcc53040dfc925d1b169fbedc575f9eec30fad4133bdd38ed8efdfc3f2969d3 contains 6 records:
Transaction Hash 0x1f4ca78710717a409b2cee42702db582b13d4e544adf116a86c605f92973d935 contains 6 records:
Transaction Hash 0x2b198f8b99cc4a183fdb5125b09e1f32521b7f5351e6b12307d5022ba030708c contains 4 records:
Transaction Hash 0x3de619b29ddab8d241f47d7a21bde8318691b71244e4658dedf97f5eba4705b3 contains 4 records:
Transaction Hash 0x3f1e3bfa355baded87207a2de30165c09a64d1ecb070f04c819587cc9386a1b0 contains 4 records:
Transaction Hash 0x3f550a50553c3033b0b0d32a435d1c0a9d3ba8323a4f9a8de1ffdcaa2cbaf739 contains 20 records:
Transaction Hash 0x427a515c5b0967335ebc0dc437bdc75ca1d2cf6d5573a6e477b399de40dcb9a6 contains 4 records:
Transaction Hash 0x52dd542f161f822026ef0050e6002c161957d086113dd7e5bf23a6e54ee02074 contains 6 records:
Transaction Hash 0x53d7b42ca96eaa61f5fd8e49624078d39c26286ef30bc5223bfdcf2ca2856a9e contains 4 records:
Transaction Hash 0x564

In [14]:
def process_duplicate_hashes(duplicate_hashes: pd.DataFrame, address: str, base_pattern) -> pd.DataFrame:
    """
    Process duplicate transaction hashes to identify BUY/SELL actions.
    : param duplicate_hashes: DataFrame containing duplicate transaction hashes.
    : param address: Ethereum address to filter transactions for.
    : param base_tokens: Set of base tokens to filter transactions for.
    """
    base_pattern_str = base_pattern.pattern
    output_records = []
    for hash_val, group in duplicate_hashes.groupby('hash'):
        # base_tokens_group = group[group['tokenSymbol'].isin(base_tokens)]
        # other_tokens_group = group[~group['tokenSymbol'].isin(base_tokens)]
        base_tokens_group = group[group['tokenSymbol'].str.contains(base_pattern_str, case=False, regex=True)]
        other_tokens_group = group[~group['tokenSymbol'].str.contains(base_pattern_str, case=False, regex=True)]

        if not base_tokens_group.empty and not other_tokens_group.empty:
            transaction_type = "SELL" if base_tokens_group['to'].iloc[0] == address else "BUY"
            base_token_value = base_tokens_group['ActualValue'].sum()
            base_token_symbol = base_tokens_group['tokenSymbol'].iloc[0]

            other_token_value = other_tokens_group['ActualValue'].sum()
            other_token_symbol = other_tokens_group['tokenSymbol'].iloc[0]

            average_price = base_token_value / other_token_value if other_token_value > 0 else 0
            price_unit = f"{base_token_symbol}/{other_token_symbol}"
            record = (
                f"{group['timeStamp'].iloc[0]} W {transaction_type} {base_token_value} {base_token_symbol} "
                f"of {other_token_value} {other_token_value} at avg price {average_price:.6f} "
                f"(at {group['dateTime'].iloc[0]})"
            )

            output_records.append({
                "formatted_record": record,
                "timeStamp": group['timeStamp'].iloc[0],
                "dateTime": group['dateTime'].iloc[0],
                "hash": group['hash'].iloc[0],
                "transaction_type": transaction_type,
                "base_token_volume": base_token_value,
                "other_token_volume": other_token_value,
                "average_price": average_price,
                "tokenSymbol": other_token_symbol,
                "price_unit": price_unit
            })
            
    return pd.DataFrame(output_records)

record_df_1 = process_duplicate_hashes(duplicate_hashes, ADDRESS, base_pattern)

if not record_df_1.empty:
    record_df_1 = record_df_1.sort_values(by='timeStamp').reset_index(drop=True)

    print("\nFormatted Transactions:")
    for record in record_df_1['formatted_record']:
        print(record)



Formatted Transactions:
1727625191 W BUY 77473.857994 aEthUSDT of 29.02304217570304 29.02304217570304 at avg price 2669.391359 (at 2024-09-29 15:53:11)
1727625203 W BUY 290228.991102 aEthUSDT of 108.7210333165128 108.7210333165128 at avg price 2669.483376 (at 2024-09-29 15:53:23)
1727625227 W BUY 33956.791958 aEthUSDT of 12.719486449708407 12.719486449708407 at avg price 2669.666900 (at 2024-09-29 15:53:47)
1727625239 W BUY 522412.183984 aEthUSDT of 195.67785536860632 195.67785536860632 at avg price 2669.756284 (at 2024-09-29 15:53:59)
1727625251 W BUY 28359.145407 aEthUSDT of 10.621994565039124 10.621994565039124 at avg price 2669.851244 (at 2024-09-29 15:54:11)
1727625263 W BUY 205569.194397 aEthUSDT of 76.99397291362723 76.99397291362723 at avg price 2669.938784 (at 2024-09-29 15:54:23)
1727625275 W BUY 16218.662388 aEthUSDT of 6.074398602380136 6.074398602380136 at avg price 2670.002983 (at 2024-09-29 15:54:35)
1727625287 W BUY 361108.988357 aEthUSDT of 135.24357024489677 135.2435

In [ ]:
from tabulate import tabulate

if not record_df_1.empty:
    unique_tokens_count = record_df_1["tokenSymbol"].nunique()
    unique_tokens = record_df_1["tokenSymbol"].unique()
    results = []

    for token in unique_tokens:
        # token_df = record_df_1[record_df_1["formatted_record"].str.contains(fr"\b{token}\b", regex=True)]
        token_df = record_df_1[record_df_1["formatted_record"].str.contains(base_pattern, regex=True)]

        print(f"{token}: {len(token_df)} records")

        pnl_df = calculate_pnl(token_df)
        print(tabulate(pnl_df, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))   
        total_pnl = pnl_df["pnl"].sum()

        results.append({"token": token, "total_pnl": total_pnl})

    pnl_summary_df = pd.DataFrame(results)

    print(tabulate(pnl_summary_df, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))
else:
    print("No transactions found")

WETH: 424 records
+---------------------+-------------+--------+----------+----------+------------+
| Date                |   timeStamp | Type   |   amount |    price |        pnl |
+=====================+=============+========+==========+==========+============+
| 2024-09-29 15:53:11 |  1727625191 | BUY    |    29.02 |  2669.39 |   20293.85 |
+---------------------+-------------+--------+----------+----------+------------+
| 2024-09-29 15:53:23 |  1727625203 | BUY    |   108.72 |  2669.48 |   76011.24 |
+---------------------+-------------+--------+----------+----------+------------+
| 2024-09-29 15:53:47 |  1727625227 | BUY    |    12.72 |  2669.67 |    8890.37 |
+---------------------+-------------+--------+----------+----------+------------+
| 2024-09-29 15:53:59 |  1727625239 | BUY    |   195.68 |  2669.76 |  136752.85 |
+---------------------+-------------+--------+----------+----------+------------+
| 2024-09-29 15:54:11 |  1727625251 | BUY    |    10.62 |  2669.85 |    7422.36 

In [20]:
matched_records = []
matched_indices = []
filtered_transaction_data_1 = filtered_transaction_data[~filtered_transaction_data['hash'].isin(duplicate_hashes['hash'])].reset_index(drop=True)

skip_next = False
for i in range(len(filtered_transaction_data_1) - 1):
    if skip_next:
        skip_next = False
        continue

    current_row = filtered_transaction_data_1.iloc[i]
    next_row = filtered_transaction_data_1.iloc[i + 1]

    if ((current_row['to'] == ADDRESS and next_row['from'] == ADDRESS) or
        (current_row['from'] == ADDRESS and next_row['to'] == ADDRESS)):
        
        if ((current_row['tokenSymbol'] in BASE_TOKENS or next_row['tokenSymbol'] in BASE_TOKENS) and
        not (current_row['tokenSymbol'] in BASE_TOKENS and next_row['tokenSymbol'] in BASE_TOKENS)):
            
            if current_row['to'] == ADDRESS:
                transaction_type = "\'SELL\'"
                base_token = current_row
                other_token = next_row
            else:
                transaction_type = "\'BUY\'"
                base_token = next_row
                other_token = current_row

            base_token_info = f"{base_token['ActualValue']} {base_token['tokenSymbol']}"
            other_token_info = f"{other_token['ActualValue']} {other_token['tokenSymbol']}"

            record = (
                f"{current_row['timeStamp']} W {transaction_type} {other_token_info} of {base_token_info} "
                f"(at {current_row['dateTime']})"
            )
            
            matched_records.append({"formatted_record": record, "dateTime": current_row['dateTime']})
            matched_indices.extend([i, i + 1])

            skip_next = True

# delete the matched data for later processing
filtered_transaction_data_2 = filtered_transaction_data_1.drop(index=matched_indices).reset_index(drop=True)

matched_df = pd.DataFrame(matched_records)

# # combine the first two dataframes and sort by time
# combined_df = pd.concat([output_df, matched_df], ignore_index=True)

if not combined_df.empty:
    combined_df = combined_df.sort_values(by='dateTime').reset_index(drop=True)
    for record in matched_df['formatted_record']:
        print(record)

1731215963 W 'BUY' 460.39922 WETH of 1470503.029812 USDT (at 2024-11-10 05:19:23)
1731215963 W 'BUY' 9.208453933839 WETH of 29411.560276 USDT (at 2024-11-10 05:19:23)
1731372791 W 'SELL' 2999.0 USDT of 897.1320453514285 WETH (at 2024-11-12 00:53:11)
1732382399 W 'SELL' 2874827.05323 USDT of 1.04124492 WBTC (at 2024-11-23 17:19:59)
1733197511 W 'BUY' 37.792758947370814 aEthWETH of 137867.98464 USDT (at 2024-12-03 03:45:11)
1733197511 W 'BUY' 32.606148929826624 aEthWETH of 118947.231296 USDT (at 2024-12-03 03:45:11)
1733197535 W 'BUY' 44.65963943859932 aEthWETH of 162918.364672 USDT (at 2024-12-03 03:45:35)
1733197535 W 'BUY' 44.84202339830328 aEthWETH of 163583.701357 USDT (at 2024-12-03 03:45:35)
1733258231 W 'BUY' 48.42034696872533 WETH of 174071.14735256755 DAI (at 2024-12-03 20:37:11)
1733258231 W 'BUY' 16.533555570276416 WETH of 59438.132275143726 DAI (at 2024-12-03 20:37:11)
1733258231 W 'BUY' 10.694595375673932 WETH of 38447.0703755478 DAI (at 2024-12-03 20:37:11)
1733258387 W 'B

In [21]:
single_records = []

for i, row in filtered_transaction_data_2.iterrows():
    if row['tokenSymbol'] in BASE_TOKENS:
        if row['from'] == ADDRESS:
            transaction_type = "single SELL"
        elif row['to'] == ADDRESS:
            transaction_type = "single BUY"
    else:
        if row['from'] == ADDRESS:
            transaction_type = "single BUY"
        elif row['to'] == ADDRESS:
            transaction_type = "single SELL"

    record = f"{row['timeStamp']} W {transaction_type} {row['ActualValue']} {row['tokenSymbol']} (at {row['dateTime']})"
    single_records.append({
        "formatted_record": record,
        "dateTime": row['dateTime'],
        "timeStamp": row['timeStamp'],
        "hash": row['hash']
    })

record_df_3 = pd.DataFrame(single_records)

if not record_df_3.empty:
    record_df_3 = record_df_3.sort_values(by='dateTime').reset_index(drop=True)
    for record in record_df_3['formatted_record']:
        print(record)


1727573591 W single SELL 52050802.583630405 SHIB (at 2024-09-29 01:33:11)
1727573939 W single SELL 2979587.067423 aEthUSDT (at 2024-09-29 01:38:59)
1727826767 W single BUY 1115.9580355467156 WETH (at 2024-10-01 23:52:47)
1731193379 W single SELL 2893195.430029 USDT (at 2024-11-09 23:02:59)
1731339503 W single SELL 2966016.159586 USDT (at 2024-11-11 15:38:23)
1731372059 W single BUY 208146.704067 aEthUSDT (at 2024-11-12 00:40:59)
1731372059 W single BUY 1202356.990403 aEthUSDT (at 2024-11-12 00:40:59)
1731372059 W single BUY 1198839.544108 aEthUSDT (at 2024-11-12 00:40:59)
1731372071 W single BUY 208146.704067 aEthUSDT (at 2024-11-12 00:41:11)
1731372083 W single BUY 142021.992291 aEthUSDT (at 2024-11-12 00:41:23)
1731373631 W single SELL 2993536.984778 USDT (at 2024-11-12 01:07:11)
1731408551 W single BUY 896.3252134286488 WETH (at 2024-11-12 10:49:11)
1731953183 W single SELL 2844562.651761 USDT (at 2024-11-18 18:06:23)
1732009475 W single BUY 916.268990125626 WETH (at 2024-11-19 09:4